In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

### 16.3 Chef Crew

In [3]:
from crewai import Crew, Agent, Task

head_chef = Agent(
    role="Head Chef",
    goal="Create ethnic cuisine recipies that are easy to cook at home.",
    backstory=""" 
    You are a famous chef that specializes in cuisine from countries all around the world.
    You know how to cook the most traditional dishes from all cultures but you also know how to adapt them for people to be able to cook them at home.
    """,
    verbose=True,
    allow_delegation=False,
)

vegan_chef = Agent(
    role="Vegan Chef",
    goal="Turn any recipe into a healthy vegetarian recipe that is easy to cook with home ingredients.",
    backstory=""" 
    You are a chef specialized in healthy cooking.
    You can take any recipe and change the ingredients to make it vegetarian friendly without loosing the escense of the dish and what makes it delicious.
    """,
    verbose=True,
    allow_delegation=False,
)

In [4]:
normal_recipe = Task(
    description="Come up with a {dish} that serves {people} people.",
    agent=head_chef,
    expected_output=""" 
    Your answer MUST have three sections, the ingredients required with their quantities, the preparation instructions and serving suggestions"
    """,
    output_file="normal_recipe.md",
)

vegan_recipe = Task(
    description="Replace the ingredients of a recipe to make it vegetarian without making it less delicious, adjust if needed.",
    agent=vegan_chef,
    expected_output=""" 
    Your answer MUST have four sections, the ingredients required with their quantities, the preparation instructions, serving suggestions and an explanation of the replaced ingredients.
    """,
    output_file="vegan_recipe.md",
)

In [6]:
crew = Crew(
    tasks=[
        normal_recipe,
        vegan_recipe
    ],
    agents=[
        head_chef,
        vegan_chef
    ],
    verbose=True,
)

result = crew.kickoff(
    inputs={
        "dish": "Greek dinner",
        "people": "5",
    }
)

 
[2025-03-11 22:14:47][🚀 CREW 'CREW' STARTED, 63F04106-E598-49F0-AFAC-98E52CB229C1]: 2025-03-11 22:14:47.567471
 
[2025-03-11 22:14:47][📋 TASK STARTED: COME UP WITH A GREEK DINNER THAT SERVES 5 PEOPLE.]: 2025-03-11 22:14:47.583484
 
[2025-03-11 22:14:47][🤖 AGENT 'HEAD CHEF' STARTED TASK]: 2025-03-11 22:14:47.585485
# Agent: Head Chef
## Task: Come up with a Greek dinner that serves 5 people.
 
[2025-03-11 22:14:47][🤖 LLM CALL STARTED]: 2025-03-11 22:14:47.586484
 
[2025-03-11 22:14:59][✅ LLM CALL COMPLETED]: 2025-03-11 22:14:59.595346


# Agent: Head Chef
## Final Answer: 
**Ingredients Required:**

*For the main dish (Greek Moussaka):*  
- 3 medium eggplants, sliced into 1/4 inch rounds  
- 1 pound ground beef or lamb  
- 1 large onion, diced  
- 2 cloves garlic, minced  
- 1 can (14 oz) crushed tomatoes  
- 1 tablespoon tomato paste  
- 1 teaspoon dried oregano  
- 1 teaspoon cinnamon  
- 1/2 teaspoon salt  
- 1/2 teaspoon black pepper  
- 3 medium potatoes, peeled and thinly sliced

### 16.5 Content Farm Crew

In [ ]:
from crewai import Crew, Agent, Task
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

researcher = Agent(
    role="Senior Researcher",
    goal="Search the web, extract and analyze information.",
    backstory=""" 
    You produce the highest quality research possible.
    You use multiple sources of information and you always double check your sources to make sure they are true and up to date.
    You want to impress your coworkers with your work.
    """,
    allow_delegation=False,
    verbose=True,
    tools=[
        search_tool,
        scrape_tool,
    ],
    max_iter=10,
)

editor = Agent(
    role="Senior Writer/Editor",
    goal="Write engaging blog posts.",
    backstory=""" 
    You write content that keeps people engaged and entertained.
    Your content is easy to read it is informative and it makes people want to share it with their friends.
    You are working for a very important client.
    """,
    verbose=True,
)

task = Task(
    description="Write a blog post about {topic}",
    agent=editor,
    expected_output=""" 
    A blog post with an introduction, at least three sub-sections of content, links to sources, a set of suggested hashtags for social media and a catchy title.
    """,
    output_file="blog_post.md",
)

crew = Crew(
    agents=[
        researcher, 
        editor,
    ],
    tasks=[
        task,
    ],
    verbose=True,
)

result = crew.kickoff(
    inputs={
        "topic": "The biggest box office flops of 2024",
    },
)

 
[2025-03-11 22:39:06][🚀 CREW 'CREW' STARTED, FBB281D3-77C5-40E8-B75E-DD547C11EDB2]: 2025-03-11 22:39:06.734770
 
[2025-03-11 22:39:06][📋 TASK STARTED: WRITE A BLOG POST ABOUT THE BIGGEST BOX OFFICE FLOPS OF 2024]: 2025-03-11 22:39:06.753943
 
[2025-03-11 22:39:06][🤖 AGENT 'SENIOR WRITER/EDITOR' STARTED TASK]: 2025-03-11 22:39:06.755943
# Agent: Senior Writer/Editor
## Task: Write a blog post about The biggest box office flops of 2024
 
[2025-03-11 22:39:06][🤖 LLM CALL STARTED]: 2025-03-11 22:39:06.755943
 
[2025-03-11 22:39:34][✅ LLM CALL COMPLETED]: 2025-03-11 22:39:34.569989


# Agent: Senior Writer/Editor
## Final Answer: 
**Title: The Unfortunate Bombs of 2024: A Look at the Biggest Box Office Flops**

The world of cinema is often a mix of dazzling successes and painful failures. As we take a closer look at the year 2024, several films have not only missed the mark—they’ve plummeted. The following exploration highlights the biggest box office flops of this year, revealing not onl

### 16.6 Pydantic Outputs

In [8]:
from pydantic import BaseModel
from typing import List

class SubSection(BaseModel):
    title: str
    content: str

class BlogPost(BaseModel):
    title: str
    introduction: str
    sections: List[SubSection]
    sources: List[str]
    hashtags: List[str]

In [ ]:
from crewai import Crew, Agent, Task
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

researcher = Agent(
    role="Senior Researcher",
    goal="Search the web, extract and analyze information.",
    backstory=""" 
    You produce the highest quality research possible.
    You use multiple sources of information and you always double check your sources to make sure they are true and up to date.
    You want to impress your coworkers with your work.
    """,
    allow_delegation=False,
    verbose=True,
    tools=[
        search_tool,
        scrape_tool,
    ],
    max_iter=10,
)

editor = Agent(
    role="Senior Writer/Editor",
    goal="Write engaging blog posts.",
    backstory=""" 
    You write content that keeps people engaged and entertained.
    Your content is easy to read it is informative and it makes people want to share it with their friends.
    You are working for a very important client.
    """,
    verbose=True,
)

task = Task(
    description="Write a blog post about {topic}",
    agent=editor,
    expected_output=""" 
    A blog post with an introduction, at least three sub-sections of content, links to sources, a set of suggested hashtags for social media and a catchy title.
    """,
    output_file="blog_post.md",
    output_json=BlogPost,
)

crew = Crew(
    agents=[
        researcher, 
        editor,
    ],
    tasks=[
        task,
    ],
    verbose=True,
)

result = crew.kickoff(
    inputs={
        "topic": "The biggest box office flops of 2024",
    },
)

 
[2025-03-11 22:51:00][🚀 CREW 'CREW' STARTED, 22B2E676-0D71-42D1-B9E3-8FB13921A276]: 2025-03-11 22:51:00.726758
 
[2025-03-11 22:51:00][📋 TASK STARTED: WRITE A BLOG POST ABOUT THE BIGGEST BOX OFFICE FLOPS OF 2024]: 2025-03-11 22:51:00.751831
 
[2025-03-11 22:51:00][🤖 AGENT 'SENIOR WRITER/EDITOR' STARTED TASK]: 2025-03-11 22:51:00.755848
# Agent: Senior Writer/Editor
## Task: Write a blog post about The biggest box office flops of 2024
 
[2025-03-11 22:51:00][🤖 LLM CALL STARTED]: 2025-03-11 22:51:00.755848
 
[2025-03-11 22:51:07][✅ LLM CALL COMPLETED]: 2025-03-11 22:51:07.407829


# Agent: Senior Writer/Editor
## Final Answer: 
{
  "title": "The Biggest Box Office Flops of 2024: Hits That Missed",
  "introduction": "As we journey through 2024, the movie industry has witnessed its fair share of high-profile releases. While some films have set the box office alight, others have fizzled out spectacularly, leaving studios and fans scratching their heads. This blog post dives into the bigge

### 16.7 Async Youtuber Crew

In [ ]:
from crewai import Crew, Agent, Task
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, YoutubeChannelSearchTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
youtube_tool = YoutubeChannelSearchTool()

researcher = Agent(
    role="Senior Researcher",
    goal="Search the web, extract and analyze information.",
    backstory=""" 
    You produce the highest quality research possible.
    You use multiple sources of information and you always double check your sources to make sure they are true and up to date.
    You want to impress your coworkers with your work.
    """,
    allow_delegation=False,
    tools=[
        search_tool,
        scrape_tool,
        youtube_tool
    ],
    max_iter=10,
    verbose=True,
)

marketer = Agent(
    role="Senior Marketer",
    goal="Come up with ideas that generate viral and useful content.",
    backstory=""" 
    You work at a marketing agency.
    You are the best at coming up with ideas to make content go viral.
    Your ideas are used for video, advertising, social media marketing, the content you produce appeals to a young audience.
    """,
    verbose=True,
)

writer = Agent(
    role="Senior Writer",
    goal="Write scripts for viral Youtube videos.",
    backstory=""" 
    You write scripts for videos that keep people engaged and entertained.
    Your content is easy and fun to watch, it is informative and it makes people want to share it with their friends.
    You are working for a very important client.
    """,
    verbose=True,
)

brainstorm_task = Task(
    description="Come up with 5 video ideas for a Youtube channel in the {industry} industry",
    agent=marketer,
    expected_output="Your answer MUST be a list of 5 ideas for a Youtube video with an explanation of what the angle of the video would be.",
    output_file="ideas_task.md",
    human_input=True,
)

selection_task = Task(
    description="Select a video idea that has the highest potential of going viral.",
    agent=writer,
    expected_output="Your answer MUST include the idea that was selected as well as an explanation of why that selection was made.",
    human_input=True,
    context=[
        brainstorm_task,
    ],
)

research_task = Task(
    description="Do all the research required to write the script of a medium length video about the selected idea.",
    agent=researcher,
    expected_output="Your answer MUST have all the information a writer would need to write a Youtube script.",
    output_file="research_task.md",
    async_execution=True,
    context=[
        selection_task,
    ],
)

competitors_task = Task(
    description="Search for videos or articles in the {industry} industry that are similar to the video idea we are working on and suggest ways our video can be different from theirs.",
    agent=researcher,
    expected_output="Your answer MUST have a list of suggestions writers can follow to make sure the video is as unique and as different from competitors as possible.",
    output_file="competitors_task.md",
    async_execution=True,
    context=[
        selection_task,
    ]
)

inspiration_task = Task(
    description="Search for videos or articles that are similar to the video idea we are working on but from other industries.",
    agent=researcher,
    expected_output="Your answer MUST have a list of examples of articles and videos that have a similar angle as the video we are making but that are in different industries.",
    output_file="inspiration_task.md",
    async_execution=True,
    context=[
        selection_task,
    ]
)

script_task = Task(
    description="Write the script for a Youtube video for a channel in the {industry} industry",
    agent=writer,
    expected_output="A script for a Youtube video with a title, an introduction, at least three sections, and an outro. Make sure to also include the prompt to generate a thumbnail for the video.",
    output_file="script_task.md",
    context=[
        selection_task,
        research_task,
        competitors_task,
        inspiration_task,
    ]
)

crew = Crew(
    agents=[
        researcher,
        writer,
        marketer,
    ],
    tasks=[
        brainstorm_task,
        selection_task,
        research_task,
        inspiration_task,
        competitors_task,
        script_task,
    ],
    verbose=True
)

result = crew.kickoff(
    inputs = {
        "industry": "Hot Sauce",
    },
)

 
[2025-03-12 17:15:15][🚀 CREW 'CREW' STARTED, D1BC25C6-0C5F-431C-A6F8-FA42F2032204]: 2025-03-12 17:15:15.577235
 
[2025-03-12 17:15:15][📋 TASK STARTED: COME UP WITH 5 VIDEO IDEAS FOR A YOUTUBE CHANNEL IN THE HOT SAUCE INDUSTRY]: 2025-03-12 17:15:15.599477
 
[2025-03-12 17:15:15][🤖 AGENT 'SENIOR MARKETER' STARTED TASK]: 2025-03-12 17:15:15.601478
# Agent: Senior Marketer
## Task: Come up with 5 video ideas for a Youtube channel in the Hot Sauce industry
 
[2025-03-12 17:15:15][🤖 LLM CALL STARTED]: 2025-03-12 17:15:15.601961
 
[2025-03-12 17:15:25][✅ LLM CALL COMPLETED]: 2025-03-12 17:15:25.438197


# Agent: Senior Marketer
## Final Answer: 
1. **"Hot Sauce Roulette Challenge"**
   - **Angle:** Set up a game of roulette where participants spin a wheel that lands on different hot sauces with varying heat levels—from mild to extremely spicy. Each person must taste the sauce they land on and share their reactions live. This will include a fun commentary about the heat level, flavor profile

### 16.8 Custom Tools

In [3]:
from crewai import Crew, Agent, Task


In [4]:
from crewai.tools import tool
import yfinance as yf

class Tools:
    @tool("One month stock price history")
    def stock_price(ticker):
        """ 
        Useful to get a month's worth of stock price data as CSV.
        The input of this tool should a ticker, for example AAPL, NET, TSLA etc...
        """
        stock = yf.Ticker(ticker)
        return stock.history(period="1mo").to_csv()
    
    @tool("Stock news URLs")
    def stock_news(ticker):
        """ 
        Useful to get URLs of news articles related to a stock.
        The input to this tool should be a ticker, for example AAPL, NET, TSLA etc...
        """
        stock = yf.Ticker(ticker)
        return list(map(lambda x: x["link"], stock.news))
    
    @tool("Company's income statement")
    def income_stmt(ticker):
        """ 
        Useful to get the income statement of a stock as CSV.
        The input to this tool should be a ticker, for example AAPL, NET, TSLA etc...
        """
        stock = yf.Ticker(ticker)
        return stock.income_stmt.to_csv()
    
    @tool("Balance sheet")
    def balance_sheet(ticker):
        """ 
        Useful to get the balance sheet of a stock as CSV.
        The input to this tool should be a ticker, for example AAPL, NET, TSLA etc...
        """
        stock = yf.Ticker(ticker)
        return stock.balance_sheet.to_csv()
    
    @tool("Get insider transactions")
    def insider_transactions(ticker):
        """ 
        Useful to get the insider transactions of a stock as CSV.
        The input to this tool should be a ticker, for example AAPL, NET, TSLA etc...
        """
        stock = yf.Ticker(ticker)
        return stock.insider_transactions.to_csv()

In [ ]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

class Agents:
    def technical_analyst(self):
        return Agent(
            role="Technical Analyst",
            goal="Analyses the movements of a stock and provides insights on trends, entry points, resistance and support levels.",
            backstory="An expert in technical analysis, you are known for your ability to predict stock movements and trends based on historical data. You provide valuable insights to your customers.",
            verbose=True,
            tools=[
                Tools.stock_price,
            ],
        )
    
    def financial_analyst(self):
        return Agent(
            role="Financial Analyst",
            goal="Uses financial statements, insider trading data, and other financial metrics to evaluate a stock's financial health and performance.",
            backstory="Your are a very experienced investment advisor who uses a combination of technical and fundamental analysis to provide strategic investment advice to your clients. You look at a company's financial health, market sentiment, and qualitative data to make informed recommendations.",
            verbose=True,
            tools=[
                Tools.balance_sheet,
                Tools.income_stmt,
                Tools.insider_transactions,
            ],
        )
    
    def researcher(self):
        return Agent(
            role="Researcher",
            goal="Gathers, interprets and summarizes vasts amounts of data to provide a comprehensive overview of the sentiment and news surrounding a stock.",
            backstory="You are skilled in gathering and interpreting data from various sources to give a complete picture of a stock's sentiment and news. You read each data source carefully and extract the most important information. Your insights are crucial for making informed investment decisions.",
            verbose=True,
            tools=[
                Tools.stock_news,
                SerperDevTool(),
                ScrapeWebsiteTool(),
            ],
        )
    
    def hedge_fund_manager(self):
        return Agent(
            role="Hedge Fund Manager",
            goal="Manages a portfolio of stocks and makes strategic investment decisions to maximize returns using insights from financial analysts, technical analysts, and researchers.",
            backstory="You are a seasoned hedge fund manager with a proven track record of making profitable investment decisions. You are known for your ability to manage risk and maximize returns for your clients.",
            verbose=True,
        )